## Dataset preprocessing



In [1]:
import numpy as nb
import pandas as pd

from sklearn import preprocessing
from sklearn import decomposition
from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot as plt

%matplotlib inline

import datetime

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tiitvaino","key":"ced67cd36a3bf255ff49cebde94923fe"}'}

In [3]:
# check if exists
!ls -lha kaggle.json

-rw-r--r-- 1 root root 65 Dec  3 11:41 kaggle.json


In [4]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c predict-electricity-consumption

  0% 0.00/174k [00:00<?, ?B/s]
100% 174k/174k [00:00<00:00, 51.7MB/s]


In [6]:
!unzip predict-electricity-consumption.zip

Archive:  predict-electricity-consumption.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


### Timestamp extraction

Because crazy things happened in the past year,  we validated that, some specific dates had significantly higher electricity prices. Therefore we do weekday, month, and time extraction from the timestamp.  



In [27]:
def extract_weekday(dataset):
    splits = dataset['date'].astype(str).str.split('-')
    dataset['weekday'] = [datetime.date(int(year), int(month), int(day)).weekday() for (year, month, day) in splits]

In [28]:
def extract_month(dataset):
    dataset['month'] = [month for (_, month, _) in dataset['date'].astype(str).str.split('-')]
    dataset['month'] = dataset['month'].astype(int)

In [29]:
def extract_datetime(dataset):
    dataset.loc[:,'time'] = pd.to_datetime(dataset.loc[:,'time'], format="%Y-%m-%d %H:%M:%S", utc=True)
    dataset['date'] = dataset['time'].dt.date
    dataset['hour'] = dataset['time'].dt.hour

In [30]:
def extract_features(dataset):
    extract_datetime(dataset)
    extract_month(dataset)
    extract_weekday(dataset)


### Feature dropping

In Estonia, there are approximately 500\-800 millimeters of rain on average. Our dataset consisted of only about 140mm of rain, which is definitely not correct. Also, the amount of snow was inappropriate for the  
 same reason.


In [31]:
def drop_features(dataset):
    dataset.drop(columns=['time','date'], inplace=True) #'prcp',


In [32]:
def replace_nan(dataset):
    dataset
    dataset['snow'] = dataset['snow'].fillna(0)

In [33]:
def drop_rows(dataset):
    # Deal with NaN values
    
    replace_nan(dataset)

    initial_len = len(dataset)
    dataset.dropna(inplace=True)
    new_len = len(dataset)
    if (initial_len != new_len):
        print(f'Dropped {initial_len - new_len} row')

    # Deal with outliners
    dataset.drop(dataset[dataset['el_price'] > 1].index , inplace=True)

In [34]:
def preprocess(dataset):
    extract_features(dataset)
    drop_features(dataset)
    drop_rows(dataset)


### Import dataset

Here we import dataset, do inital processing and split into train and validation.

In [35]:
def read_dataset(file_name) -> pd.DataFrame:
    return pd.read_csv(file_name)

In [36]:
def extract_labels(dataset) -> (pd.DataFrame, pd.Series):
    X_train = dataset.loc[:, ~dataset.columns.isin(['consumption'])]
    y_train = dataset['consumption']
    return (X_train, y_train)

In [40]:
train_df = read_dataset('train.csv')

In [43]:
train_df.prcp.value_counts()

0.0    2043
0.2      20
0.1      16
0.3      15
0.7       6
1.2       6
0.6       5
0.4       4
0.5       4
1.0       4
2.7       3
3.6       2
0.9       2
2.2       2
1.8       2
1.6       2
2.4       2
1.3       2
1.1       2
0.8       2
2.0       1
3.0       1
4.2       1
3.1       1
2.9       1
4.4       1
3.9       1
1.7       1
3.3       1
6.0       1
1.4       1
1.5       1
1.9       1
3.5       1
7.9       1
Name: prcp, dtype: int64

In [ ]:
preprocess(train_df)
X_train, y_train = extract_labels(train_df)

In [38]:
X_train.shape

(1962, 14)

In [39]:
X_test = read_dataset('test.csv')
preprocess(X_test)
print(X_test.shape)

(168, 14)


In [44]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [45]:
X_train.shape

(1569, 14)

In [46]:
import itertools

# Numpy
import numpy as np

# XGBoost
import xgboost as xgb

# Pandas
import pandas as pd

# Sklearn
import sklearn.preprocessing
import sklearn.utils
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Visualiseerimine
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
#reg = xgb.XGBRegressor(tree_method="gpu_hist")
# Fit the model using predictor X and response y.

In [48]:
# XGBoosts wants data to be wrapped into special formats
#dtrain = xgb.DMatrix(X_train,label=y_train)
#dval = xgb.DMatrix(X_val,label=y_val)
#dtest = xgb.DMatrix(X_test_reduced)

In [49]:
xgbr = xgb.XGBRegressor(verbosity=0)

In [50]:
xgbr.fit(X_train, y_train)

XGBRegressor(verbosity=0)

In [51]:
yvalpred = xgbr.predict(X_val)
mse = mean_squared_error(y_val, yvalpred)
mae = mean_absolute_error(y_val, yvalpred)
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

MSE: 0.30
MAE: 0.39


In [52]:
prediction = xgbr.predict(X_test)

In [53]:
prediction

array([0.32050276, 0.34290165, 0.2723465 , 0.2424058 , 0.76022065,
       0.3405425 , 0.28681597, 0.45490944, 0.64308965, 0.5554238 ,
       0.55885994, 0.60164666, 0.50343853, 0.50343853, 0.7044125 ,
       0.6891379 , 0.68322235, 0.77750623, 0.8908756 , 0.8908756 ,
       0.8908756 , 0.8908756 , 0.92901057, 0.42334387, 0.5044315 ,
       0.3070174 , 0.29580486, 0.21297818, 0.681802  , 0.35142583,
       0.52166873, 0.37343913, 0.57301295, 0.5858222 , 0.7326449 ,
       0.70568264, 0.7445247 , 0.7081604 , 0.76231176, 0.77798474,
       0.80402994, 0.9602808 , 0.9468429 , 0.97936726, 0.76932985,
       0.71843195, 0.43475667, 0.38537502, 0.3619548 , 0.3238266 ,
       0.34921938, 0.2403019 , 0.6848427 , 0.40610644, 0.40610644,
       0.40610644, 0.581035  , 0.60856557, 0.6291005 , 0.6021298 ,
       0.751452  , 0.7584932 , 0.82705784, 0.82837737, 0.84273076,
       0.89106363, 0.9479465 , 0.9479855 , 0.9052729 , 0.9052729 ,
       0.49694598, 0.3661512 , 0.39337265, 0.34711462, 0.34225

In [54]:
X_test = read_dataset('test.csv')

In [56]:
predictions_dict = {'time':X_test.time,'consumption':prediction}
pred_df = pd.DataFrame(predictions_dict)
pred_df.to_csv('submission_Xgboost_v4.csv',index=False)

In [57]:
!ls

kaggle.json			     sample_submission.csv	train.csv
predict-electricity-consumption.zip  submission_Xgboost_v4.csv
sample_data			     test.csv


In [59]:
!kaggle competitions submit predict-electricity-consumption -f 'submission_Xgboost_v4.csv' -m "with out dropping prcp"

100% 6.00k/6.00k [00:02<00:00, 2.18kB/s]
Successfully submitted to Journey to zero - Predict electricity consumption